In [36]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

First, we are going to start working on modelling the states without the gdp index. We will try to keep to approach as modular as possible, in case we need to change the selected dataframe.

In [37]:
correlation_matrix = abs(pd.read_csv('Correlation_matrix.csv'))*10

We will always take the absolute value of the round of 10 correlation between states. First let us focus on swing states. We will need to select the swing state based on sources **previous to the 2020 election**. Source selected: https://fr.wikipedia.org/wiki/Swing_state#Historique (last modification: 24 october 2020).  

#### Construct the data

One big problem will be the way we will construct the validation set. It needs to be from the swing state considered. How can we prevent the creation of a deterministic validation set while imposing such a constraint ? 

###### Getting a list of states and swing states

In [38]:
swing_states = ['Texas', 'Florida', 'Ohio', 'Georgia', 'North Carolina', 'Arizona', 'Iowa', 'Pennsylvania', 'Michigan', 'Virginia', 'Minnesota', 'Wisconsin', 'Colorado', 'Nevada', 'New Hampshire']
all_states = []
index_swing_states = []
for i, file in enumerate(os.listdir('states/')):
    state = file.split('.')[0]
    all_states.append(state)
    if state in swing_states:
        index_swing_states.append(i)

##### Construct the base dataframe

In [39]:
final_df = pd.concat([pd.read_csv('states/'+file) for file in os.listdir('states/')])

##### Important modification to prevent information leakage

In [40]:
final_df = final_df[final_df['republican']==1]

In [41]:
final_df

,month_10,month_11,month_9,republican,Year,Rep_House_Prop,State,Result,rep_loyalty,popular_vote_percentage,density,RDI
0,56.380492,56.679982,53.953977,1,1988,0.285714,Alabama,1,0.8,53.370,76.757225,52087.05
3,42.175509,41.956795,44.865857,1,1992,0.285714,Alabama,1,0.9,37.450,78.959026,56035.35
5,45.410314,46.671015,45.734410,1,1996,0.428571,Alabama,1,1.0,40.720,81.841997,60091.59
7,52.511963,53.115390,51.697343,1,2000,0.714286,Alabama,1,1.0,47.870,84.927856,64078.92
8,57.096244,56.892984,56.020586,1,2004,0.714286,Alabama,1,1.0,50.730,86.426359,68482.42
...,...,...,...,...,...,...,...,...,...,...,...,...
5,44.975058,45.307159,45.169505,1,1996,1.000000,Wyoming,1,1.0,40.720,4.907941,7673.05
7,59.684991,61.311713,58.457317,1,2000,1.000000,Wyoming,1,1.0,47.870,5.053262,8155.73
9,58.828129,60.276110,60.148473,1,2008,1.000000,Wyoming,1,1.0,45.660,5.582234,9154.98
10,57.320477,60.283224,57.678128,1,2016,1.000000,Wyoming,1,1.0,46.090,5.985616,10109.96


In [42]:
final_importance_dfs = {}
for index_state in index_swing_states:  # model only for the swing states
    data_swing_state = []
    for i, state in enumerate(all_states):  
        if state == all_states[index_state]:
            continue
        if state != all_states[index_state]:   # for all the state but the swing state, remove the 2020 election
            data_state = final_df[final_df['State'] == state]
            data_state = data_state.iloc[:-2, :]
        correlation = int(correlation_matrix.iloc[index_state, i])
        if correlation > 0:  # add correlation * the dataframe for the state (where we removed the 2020 election)
            data_state_importance = pd.concat([data_state for _ in range(correlation)])
            if data_state.values.tolist():
                try:
                    data_swing_state = pd.concat([data_swing_state, data_state_importance])
                except:
                    data_swing_state = data_state_importance
    final_importance_dfs[all_states[index_state]] = data_swing_state
# for every swing state, the corresponding values are missing, this is in order to help for the validation

In [43]:
final_importance_dfs['Texas']

,month_10,month_11,month_9,republican,Year,Rep_House_Prop,State,Result,rep_loyalty,popular_vote_percentage,density,RDI
0,56.380492,56.679982,53.953977,1,1988,0.285714,Alabama,1,0.8,53.37,76.757225,52087.05
3,42.175509,41.956795,44.865857,1,1992,0.285714,Alabama,1,0.9,37.45,78.959026,56035.35
5,45.410314,46.671015,45.734410,1,1996,0.428571,Alabama,1,1.0,40.72,81.841997,60091.59
7,52.511963,53.115390,51.697343,1,2000,0.714286,Alabama,1,1.0,47.87,84.927856,64078.92
8,57.096244,56.892984,56.020586,1,2004,0.714286,Alabama,1,1.0,50.73,86.426359,68482.42
...,...,...,...,...,...,...,...,...,...,...,...,...
0,50.103222,57.584084,44.480604,1,1988,1.000000,Wyoming,1,1.0,53.37,4.754759,7002.49
3,33.404081,34.254117,34.109462,1,1992,1.000000,Wyoming,1,1.0,37.45,4.738300,7243.74
5,44.975058,45.307159,45.169505,1,1996,1.000000,Wyoming,1,1.0,40.72,4.907941,7673.05
7,59.684991,61.311713,58.457317,1,2000,1.000000,Wyoming,1,1.0,47.87,5.053262,8155.73


#### Modelling part : swing state

Enter the swing state you wish to study

In [44]:
swing_state = 'Texas'

In [45]:
def split(initial_df, final_importance_df, swing_state): # particular state missing
    swing_state_data = initial_df[initial_df['State'] == swing_state]  # data to be protected
    data_test = swing_state_data[swing_state_data['Year'] == 2020].iloc[:1, :] # test data
    X_test, y_test = data_test.drop('Result', axis=1), data_test['Result'] # X test and test labels  
    swing_state_data = swing_state_data[swing_state_data['Year'] != 2020] # we remove the test data
    X_train, X_val = train_test_split(swing_state_data, train_size = 0.7)
    X_val, y_val = X_val.drop('Result', axis=1), X_val['Result']
    df_corr_to_concat = pd.concat([X_train for _ in range(10)])
    X_values = pd.concat((final_importance_df[final_importance_df['State'] != swing_state], df_corr_to_concat))
    X_values = X_values.sample(frac=1, replace=False) 
    X, y = X_values.drop('Result', axis=1), X_values['Result']
    return X, X_val, X_test, y, y_val, y_test

In [46]:
X_train, X_val, X_test, y_train, y_val, y_test = split(final_df, final_importance_dfs[swing_state], swing_state)

In [61]:
def modelling(X_train, X_val, y_train, y_val, models, scaler=True, Poly=1, PCA_comp=0):
    # pca is the total variance explained required. 0 means that we do not want to perform pca
    if 'State' in X_train.columns:
        X_train = X_train.drop('State', axis=1)
        X_val = X_val.drop('State', axis=1)
    if 'Year' in X_train.columns:
        X_train = X_train.drop('Year', axis=1)
        X_val = X_val.drop('Year', axis=1)
    if 'republican' in X_train.columns:
        X_train = X_train.drop('republican', axis=1)
        X_val = X_val.drop('republican', axis=1)
    if PCA_comp > 0 and not scaler:
        raise ValueError('We must normalize before performing PCA')
    # if PCA_comp > 1 and Poly > 1:
        # print('Performing Polynomial Transformation on top of PCA ...')
    if scaler:
        # print('Normalizing the data ...')
        scaler = MinMaxScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_val = scaler.transform(X_val)
    if PCA_comp > 0:
        # print('Performing PCA ...')
        pca = PCA(n_components=X_train.shape[1])
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        X_val = pca.transform(X_val)
        total_variance = np.cumsum(pca.explained_variance_ratio_)
        for i, variance in enumerate(total_variance):
            if variance > PCA_comp:
                break 
        X_train, X_val = X_train[:, :i+1], X_val[:, :i+1]
    if Poly > 1:
        # print('Polynomial transformation ...')
        poly_features = PolynomialFeatures(degree=Poly, include_bias=False).fit(X_train)
        X_train = poly_features.transform(X_train)
        X_val = poly_features.transform(X_val)
    accuracies = []
    auc = []
    # print('Modelling and gathering the predictions ..')
    for i, model in enumerate(models):
        model.fit(X_train, y_train)
        predictions = model.predict(X_val)
        accuracies.append(accuracy_score(y_val, predictions))
        try:
            auc.append(roc_auc_score(y_val, predictions))
        except ValueError:
            auc.append(1)
    return models, accuracies, auc, predictions

In [62]:
models, accuracies, auc, predictions = modelling(X_train, X_val, y_train, y_val, models=[LogisticRegression()], scaler=True, Poly=2, PCA_comp=0.9)

##### Tying it all together for the modelling framework

In [71]:
models_considered = [LogisticRegression(), DecisionTreeClassifier()]
F1, acc, auc = [], [], []
for state in swing_states:
    # inspecting that everything is okay 
    X_train, X_val, X_test, y_train, y_val, y_test = split(final_df, final_importance_dfs[state], state)
    models, accuracies, aucs, predictions = modelling(X_train, X_val, y_train, y_val, models=models_considered, scaler=True, Poly=2, PCA_comp=0.9)
    acc.append(accuracies)
    auc.append(aucs)
acc_mean, auc_mean = np.mean(np.array(acc), axis=0), np.mean(np.array(auc), axis=0)
print('The best accuracy over all of the swing states is ' + str(np.max(acc_mean)) + ' and is obtained with ' + str(models_considered[np.argmax(acc_mean)]))
print('The best auc over all of the swing states is ' + str(np.max(auc_mean)) + ' and is obtained with ' + str(models_considered[np.argmax(auc_mean)]))

The best accuracy over all of the swing states is 0.7333333333333333 and is obtained with LogisticRegression()
The best auc over all of the swing states is 0.8666666666666667 and is obtained with LogisticRegression()
